<h1>ProtCB-Bind</h1>
<div>
    Before Running please update the location of the classifiers if path has changed.
    <br/>
    The predictor can be utilised by calling the function: ProtCB_Pred. 
    The predictor function takes the 4 parameters: 
    <ol>
        <li>svm_features</li>
        <li>etc_features</li>
        <li>sgd_features</li>
        <li>labels</li>
    </ol>
    The function returns the following:
    <ol>
        <li>pred_prob</li>
        <li>pred</li>
        <li> tn</li>
        <li>fn</li>
        <li>tp</li>      
        <li>fp</li>
        <li>sensitivity</li>
        <li>specificity</li>        
        <li>auc</li>
        <li>acc</li>        
        <li>mcc</li>
        <li>bacc</li>
    </ol>
</div>

In [ ]:
import sys
import numpy as np
import pandas as pd
import pickle

In [ ]:
#Get Required Notebooks and run them
%run ../Functions/PerformanceEvaluation.ipynb
%run ../Functions/Excel_Functions.ipynb

In [ ]:
#Load Pre-trained Classifiers

file_path = '../../Classifiers/SVM.pkl' #Path to SVM model
with (open(file_path, "rb")) as openfile:
  svm = pickle.load(openfile)

file_path = '../../Classifiers/ETC.pkl' #Path to ETC model
with (open(file_path, "rb")) as openfile:
  etc = pickle.load(openfile)

file_path = '../../Classifiers/SGD.pkl' #Path to SGD model
with (open(file_path, "rb")) as openfile:
  sgd = pickle.load(openfile)

In [ ]:
def ProtCB_Bind(svm_features, etc_features, sgd_features, test_labels):
    try:
        #Get Probabilites of test data
        svm_prob = svm.predict_proba(svm_features)
        etc_prob = etc.predict_proba(etc_features)
        sgd_prob = sgd.predict_proba(sgd_features)
    
        #add the features together
        add = np.add(svm_prob, etc_prob)
        add = np.add(add, sgd_prob)

        #get prediction probability and label 
        pred_prob = np.divide(add, 3)
        pred = np.argmax(pred_prob, axis=-1)
    
        tn, fn, tp, fp, sensitivity, specificity, auc_m, acc, mcc, bacc = Evaluate(pred_prob, pred, test_labels)

        return pred_prob, pred, tn, fn, tp, fp, sensitivity, specificity, auc_m, acc, mcc, bacc
    except Exception as err:
        print("An exception as occured and has been logged")
        print(err)